In [ ]:
import keras as k
import scipy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.models import Sequential
from keras.layers import Dense, Activation
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
home = '/home/nandhini/aozel/periodicBox/alpp0.1/CFD/0.6/'
fsize = 9

In [ ]:
alpp_file = 'barAlpp' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
alpp = pd.read_csv(home+alpp_file,skiprows=21,skipfooter=33)
numpy_alpp = alpp.values

Us_file = 'tildeUs' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
Us = pd.read_csv(home+Us_file,skiprows=21,skipfooter=33)
Us = Us.values
numpy_Us = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in Us])

Uf_file = 'tildeUf' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
Uf = pd.read_csv(home+Uf_file,skiprows=21,skipfooter=33)
Uf = Uf.values
numpy_Uf = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in Uf])

In [ ]:
numpy_Uslip = numpy_Uf - numpy_Us

In [ ]:
numpy_alpp.shape, numpy_Us.shape, numpy_Uf.shape, numpy_Uslip.shape

In [ ]:
betaNum_file = 'normalizedEulerianDragCoeffNum' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
betaNum = pd.read_csv(home+betaNum_file,skiprows=19,skipfooter=4)
betaNum = betaNum.values
numpy_betaNum = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in betaNum])

In [ ]:
betaDom_file = 'normalizedEulerianDragCoeffDom' + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
betaDom = pd.read_csv(home+betaDom_file,skiprows=19,skipfooter=4)
betaDom = betaDom.values
numpy_betaDom = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in betaDom])

In [ ]:
numpy_betaNum.shape, numpy_betaDom.shape

In [ ]:
numpy_ratio = numpy_betaNum / numpy_betaDom
numpy_ratio.shape

In [ ]:
original_table = np.concatenate((numpy_alpp,numpy_Uslip ),axis=1)

In [ ]:
original_table.shape

In [ ]:
nx = 80
ny = 80
nz = 320
dx = 1

In [ ]:
index1=0
npData=4

myData = np.zeros( ((nx-2)*(ny-2)*(nz-2), npData * (2*dx +1)**3) );
target = np.zeros((nx-2)*(ny-2)*(nz-2))
for i in range(1,nx-1):
    for j in range(1,ny-1):
        for k in range(1,nz-1):
            
            
            index2 = 0;
            for ii in range(-dx,dx+1):
                for jj in range(-dx,dx+1):
                    for kk in range(-dx,dx+1):
                        stencil= i+ii + (j+jj)*nx + (k+kk)*nx*ny;
                        target[index1] = numpy_ratio[stencil,-1]
                        temp = original_table[stencil][:]
                        for ind in range(npData):
                            myData[index1,index2] = temp[ind];
                            index2 = index2 + 1
            index1 = index1 +1                   
    print(i)

In [ ]:
myData.shape, target.shape

# Start here

In [ ]:
pickle.dump( myData, open( "myDataUslip"+str(fsize)+".pickle", "wb" ) )
pickle.dump( target, open( "targetUslip"+str(fsize)+".pickle", "wb" ) )

In [ ]:
myData = pickle.load( open( "myDataUslip"+str(fsize)+".pickle", "rb" ) )
target = pickle.load( open( "targetUslip"+str(fsize)+".pickle", "rb" ) )

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(myData, target, test_size=0.33, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
rf = GradientBoostingRegressor()

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
y_pred

In [ ]:
plt.hist(y_pred,1000,normed=True);
plt.xlim([-1,1])

In [ ]:
scipy.stats.pearsonr(y_realTT.reshape(-1), y_predict.reshape(-1))

In [ ]:
error_percent = (y_pred.reshape(-1) - y_test.reshape(-1))/y_test.reshape(-1)
errpd = plt.hist(error_percent,4000,normed=True)
plt.xlim([-10, 10])

In [ ]:
pickle.dump( errpd, open( "errpd"+str(fsize)+".pickle", "wb" ) )

In [ ]:
errpd_5 = pickle.load( open( "errpd"+str(5)+".pickle", "rb" ) )
errpd_9 = pickle.load( open( "errpd"+str(9)+".pickle", "rb" ) )

In [ ]:
model1_5 = np.loadtxt('pdf_errVdrift5X5X5')
model2_5 = np.loadtxt('pdf_errKbbMod5X5X5')
model1_9 = np.loadtxt('pdf_errVdrift9X9X9')
model2_9 = np.loadtxt('pdf_errKbbMod9X9X9')

In [ ]:
plt.plot(errpd_5[1][1:]+(errpd_5[1][1]-errpd_5[1][0])/2,errpd_5[0],'b')
plt.plot(model1_5[:,0], model1_5[:,1],'g')
plt.plot(model2_5[:,0], model2_5[:,1],'r')

plt.plot(errpd_9[1][1:]+(errpd_9[1][1]-errpd_9[1][0])/2,errpd_9[0],'--b')
plt.plot(model1_9[:,0], model1_9[:,1],'--g')
plt.plot(model2_9[:,0], model2_9[:,1],'--r')

plt.xlim([-1, 1])


In [ ]:
mean, cov = [0, 2], [(1, .5), (.5, 1)]
x, y = np.random.multivariate_normal(mean, cov, size=50).T
ax = sns.kdeplot(x)

In [ ]:
sns.kdeplot(error_percent)

In [ ]:
ah[0].shape

In [ ]:
plt.scatter(ah[1],ah[0])

In [ ]:
from scipy.stats.kde import gaussian_kde
from numpy import linspace

# this create the kernel, given an array it will estimate the probability over that values
kde = gaussian_kde( error_percent )
# these are the values over wich your kernel will be evaluated
dist_space = linspace( min(error_percent), max(error_percent), 100 )
# plot the results
plt.plot( dist_space, kde(dist_space) )
plt.xlim([-1,1])

In [ ]:
#Us_2 = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in Us])

In [ ]:
#Us[-1]

In [ ]:
#Us_2[-1]